In [ ]:
# # IMDB Review Dataset  Loading

# from tensorflow.keras.datasets import imdb
# import tensorflow as tf
# (x_data_train, t_data_train) , (x_data_test, t_data_test) = \
# imdb.load_data(num_words=500)


In [ ]:
# print(x_data_train.shape) # (25000,)
# print(t_data_train.shape) # (25000,)

# from tensorflow.keras.preprocessing.sequence import pad_sequences

# x_data_train_seq = pad_sequences(x_data_train, maxlen=100)

# print(x_data_train_seq.shape)  # (25000, 100)
# # 모든 Review의 길이가 100개의 token으로 획일화!
# # 각 Review에서 100개가 넘어가는 token은 삭제, 모자라는 token은 padding처리
# print(x_data_train_seq[0])

# # 결국은 one-hot encoding으로 처리해야 해요!
# # 하나하나의 token이 500개짜리 1차원 ndarray로 표현되어야 해요!
# # 전체데이터는 (100, 500)

# from tensorflow.keras.utils import to_categorical

# x_data_train_onehot = to_categorical(x_data_train_seq)
# print(x_data_train_onehot.shape)


In [ ]:
 #필요한 사항들에대해 알아보았으니..
# 이제 전체코드를 한번 작성해 보아요!

# 필요한 module import

import numpy as np
import matplotlib.pyplot as plt
# 원래 의미는 train과 test로 분리하는거지만..
# 우리는 train과 validation 데이터를 분리하는 용도로 사용!
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Data Loading
(x_data_train, t_data_train) , (x_data_test, t_data_test) = \
imdb.load_data(num_words=500)

# 2. Data 확인
print(x_data_train.shape)  # (25000,)

# 3. Data 분리(train data와 validation data)
x_data_train, x_data_valid, t_data_train, t_data_valid = \
train_test_split(x_data_train,
                 t_data_train,
                 test_size=0.2,
                 random_state=42,
                 stratify=t_data_train)

# review의 길이가 제각각인데 이거 하나의 길이로 재단해야 해요!
x_data_train_seq = pad_sequences(x_data_train, maxlen=100)
x_data_valid_seq = pad_sequences(x_data_valid, maxlen=100)

# x_data를 one-hot형태로 변형해야 입력으로 사용이 가능!
x_data_train_onehot = to_categorical(x_data_train_seq)
x_data_valid_onehot = to_categorical(x_data_valid_seq)

In [ ]:
import tensorflow as tf

# GPU 장치 목록을 가져옴
physical_devices = tf.config.list_physical_devices('GPU')

# 가상 장치 구성을 정의
virtual_device_config = tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)

# 각 GPU 장치에 대해 가상 장치 구성을 설정
for device in physical_devices:
    tf.config.experimental.set_virtual_device_configuration(device, [virtual_device_config])
# Model 구현

model = Sequential()

model.add(SimpleRNN(units=8,
                    activation='tanh',
                    input_shape=(100,500)))

model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

es_cb = EarlyStopping(monitor='val_loss',
                      patience=4,
                      verbose=1,
                      restore_best_weights=True)

cp_cb = ModelCheckpoint(filepath='./best-simplernn-model.ckpt',
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True)

history = model.fit(x_data_train_onehot,
                   t_data_train,
                   epochs=100,
                   batch_size=64,
                   validation_data=(x_data_valid_onehot,t_data_valid),
                   callbacks=[es_cb, cp_cb])

In [ ]:
train_loss = history.history['loss']
valid_loss = history.history['val_loss']

plt.plot(train_loss, 'o', color='r')
plt.plot(valid_loss, color='b')

plt.show()

In [ ]:
# 이번에는 one-hot이 아니라 word embedding방식으로 구현해 보아요!

# 필요한 module import

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Data Loading
(x_data_train, t_data_train) , (x_data_test, t_data_test) = \
imdb.load_data(num_words=500)

# 2. Data 확인
print(x_data_train.shape)  # (25000,)

# 3. Data 분리(train data와 validation data)
x_data_train, x_data_valid, t_data_train, t_data_valid = \
train_test_split(x_data_train,
                 t_data_train,
                 test_size=0.2,
                 random_state=42,
                 stratify=t_data_train)

# review의 길이가 제각각인데 이거 하나의 길이로 재단해야 해요!
x_data_train_seq = pad_sequences(x_data_train, maxlen=100)
x_data_valid_seq = pad_sequences(x_data_valid, maxlen=100)

# Model 구현

model = Sequential()

model.add(Embedding(input_dim=500,
                    output_dim=16,
                    input_length=100))

model.add(SimpleRNN(units=8,
                    activation='tanh'))

model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

es_cb = EarlyStopping(monitor='val_loss',
                      patience=4,
                      verbose=1,
                      restore_best_weights=True)

cp_cb = ModelCheckpoint(filepath='best-simplernn-model.ckpt',
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True)

history = model.fit(x_data_train_seq,
                   t_data_train,
                   epochs=100,
                   batch_size=64,
                   validation_data=(x_data_valid_seq,t_data_valid),
                   callbacks=[es_cb, cp_cb])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
    '강감찬은 나를 너무 너무 좋아해',
    '강감찬은 영화를 좋아해'
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

#단어 인덱스 확인
print(f'단어 인덱스는 : {tokenizer.word_index}')

word_encoding = tokenizer.texts_to_sequences(sentences)
print(word_encoding)

In [ ]:
# 만약 단어사전에 없는 새로운 단어가 등장하면 어떻게 되나요?
new_sentence = [
    '강감찬은 신사임당을 너무 좋아해'
]
new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
print(new_word_encoding)

# 기본적인 동작은 이렇게 단어사전에 없는 데이터는 변환이 안되기 때문에
# 그냥 없애버려요!
# 이렇게 처리하면 곤란해요! OOV(out of vocabulary) token은
# 따로 처리해줘야 해요!


In [ ]:
sentences = [
    '강감찬은 나를 너무 너무 좋아해',
    '강감찬은 영화를 좋아해'
]

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

#단어 인덱스 확인
print(f'단어 인덱스는 : {tokenizer.word_index}')

word_encoding = tokenizer.texts_to_sequences(sentences)
print(word_encoding)
new_sentence = [
    '강감찬은 신사임당을 너무 좋아해'
]
new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
print(new_word_encoding)

In [ ]:
# 위처럼 vocabulary를 구성하면 모든 token들이 포함되는 단어사전이 만들어져요!
# 이렇게 하면 단어사전이 너무 커지는 문제가 발생해요!
# 일반적으로 빈도수가 높은 단어들만 단어사전에 포함시켜요!
# 빈도수가 높은 순서로 최대 사전개수를 정하고 빈도수가 적은 단어를 제외하면 되요!

from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
    '강감찬은 나를 너무 너무 좋아해',
    '강감찬은 영화를 좋아해'
]

tokenizer = Tokenizer(
                      oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

# 단어 인덱스를 확인해보아요!
print(f'단어 인덱스는 : {tokenizer.word_index}')

# 숫자로 변형하려면 어떻게 해야 하나요?
word_encoding = tokenizer.texts_to_sequences(sentences)
print(word_encoding)

new_sentence = [
    '강감찬은 신사임당을 너무 좋아해'
]
new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
print(new_word_encoding)
from tensorflow.keras.preprocessing.sequence import pad_sequences

padding = pad_sequences(word_encoding, maxlen=4)
print(padding)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Naver가 제공하는 네이버 영화 리뷰 데이터를 로딩해 보아요!
train_file = tf.keras.utils.get_file(
    'ratings_train.txt',
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',
    extract=True
)

train_df = pd.read_csv(train_file,sep='\t')
display(train_df)

test_file = tf.keras.utils.get_file(
    'ratings_test.txt',
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',
    extract=True
)
test_df = pd.read_csv(test_file,sep='\t')
display(test_df)

In [ ]:
cnt = train_df['label'].value_counts()
print(cnt)

In [ ]:
from konlpy.tag import Okt, Komoran, Kkma

kkam = Kkma()
okt = Okt()
komoran = Komoran()

text = '이것은 소리없는 아우성. 저 푸른 해원을 향하여 흔드는 노스텔지어의 손수건'

print(kkam.morphs(text))

print(okt.morphs(text))

print(komoran.morphs(text))

In [ ]:
# 데이터 전처리도 진행해야 해요!

# 1. 댓글데이터에는 특수문자, 숫자 이런거 함께 포함되어 있어요!
#    정규식을 사용해서 영어, 한글, 띄어쓰기만 남기고 나머지 특수문자는 제거!

# 2. 결측치 제거(5개)

# 3. stopword(불용어)를 제거해야 해요!

# 4. 단어사전을 만들고 숫자로 변경하는 token화 작업을 진행하면 되요!

# 5. 동일한 문장 길이로 정리하면 되죠! (padding처리)

##############################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Naver가 제공하는 네이버 영화 리뷰 데이터를 로딩해 보아요!
train_file = tf.keras.utils.get_file(
    'ratings_train.txt',
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',
    extract=True
)
train_df = pd.read_csv(train_file, sep='\t')

test_file = tf.keras.utils.get_file(
    'ratings_test.txt',
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',
    extract=True
)
test_df = pd.read_csv(test_file, sep='\t')

# 데이터를 다운로드 해서 Df로 생성함

train_df['document'] = train_df['document'].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]','')
#display(train_df.head())
train_df = train_df.dropna()
print(train_df.shape)

# 3. stopword(불용어)처리
#    관서, 전치사, 조사, 접속사등의 의미가 없는 단어를 불용어라 하는데
#    당연히 처리해야 해요! 한글에는 이런 불용어가 정말 어마어마하게 많아요!
#    그걸 다 처리하는 건 불가능, 하지만 많이 처리하면 할 수록 성능 상승

from konlpy.tag import Okt

okt = Okt()

def word_tokenization(text):
    stop_words = ['는', '을', '를', '이', '가', '의', '던', '고', '하', '다', 
                  '은', '에', '들', '지', '게', '도']
    return [word for word in okt.morphs(text) if word not in stop_words]

data_train = train_df['document'].apply(lambda x:word_tokenization(x))

In [32]:
# 불용어 처리가 됬다고 가정하고 진행해야 될꺼 같아요!

# 단어사전을 만들고 문자를 숫자로 변경하는 token화 진행!
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_train)  # 단어사전 생성!
# 일단 총 단어개수가 몇개인지 알아보아요!
#print(f'총 단어개수는 : {len(tokenizer.word_index)}')

#print(tokenizer.word_index)

#print(tokenizer.word_counts)
# 각 토큰이 나타난 수(빈도)에서 숫자만 출력해 보아요!
print(tokenizer.word_counts.values())

# 단어 빈도수가 5회 이상인 단어가 몇개인지 확인!
def get_voca_size(threshold):
    count = 0
    for x in tokenizer.word_counts.values():
        if x > threshold:
            count += 1
    return count

voca_size = get_voca_size(5)
print(voca_size)  # 19338


odict_values([4157, 573, 8338, 23, 375, 256, 572, 4724, 424, 50651, 1241, 142, 6328, 242, 26, 3, 670, 69, 1, 75, 20, 10, 1190, 11045, 16, 2174, 11, 1064, 3921, 2834, 6315, 40, 222, 2, 5673, 2, 24, 64, 6985, 31, 1, 541, 4, 2, 206, 50, 38, 332, 20, 237, 3, 602, 1723, 116, 121, 6, 1, 2462, 1934, 206, 433, 1061, 1253, 832, 796, 2, 1043, 1640, 781, 324, 8, 4, 882, 2005, 126, 698, 1991, 1174, 9752, 301, 7849, 689, 444, 510, 2, 49, 11, 8704, 151, 5063, 937, 5299, 4694, 4837, 345, 207, 3439, 2398, 5, 2510, 631, 131, 1, 491, 2830, 652, 143, 714, 1, 11154, 3, 203, 681, 27, 1598, 4942, 324, 675, 2, 24, 690, 66, 944, 754, 6, 420, 67, 298, 21, 78, 1917, 277, 1, 1, 4279, 40, 4, 318, 312, 39, 152, 1402, 12, 5, 1056, 547, 181, 333, 139, 3553, 1791, 263, 1, 25, 6302, 274, 36, 18, 107, 794, 4039, 3732, 1391, 790, 219, 4881, 2095, 5335, 76, 115, 178, 65, 153, 349, 91, 2489, 2088, 13, 15, 168, 1874, 13, 398, 1793, 18, 24, 149, 690, 15, 1, 22, 1885, 1065, 157, 21, 6, 183, 5480, 1898, 480, 5125, 531, 512, 1

In [35]:
# 단어사전을 생성합니다.
tokenizer = Tokenizer(oov_token='<OOV>',
                      num_words=15000)
tokenizer.fit_on_texts(data_train)

#print(tokenizer.word_index)

data_train_seq = tokenizer.texts_to_sequences(data_train)

print(data_train_seq[0])
print(data_train_seq[1])

[42, 428, 8, 6671, 635]
[905, 430, 34, 576, 2, 183, 1547, 14, 957, 6109, 1]


In [37]:
max_length = max([len(x) for x in data_train_seq])
print(max_length)

from tensorflow.keras.preprocessing.sequence import pad_sequences

x_data_train = pad_sequences(data_train_seq,
                             maxlen=max_length)

t_data_train = train_df['label'].values
print(x_data_train[:1])  # padding 처리 확인!

71
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0   42  428    8 6671
   635]]


In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

model = Sequential()
model.add(Embedding(input_dim=15000,
                    output_dim=32,
                    input_length=71))
#양방향 LSTM (학습이 조금 더 잘되게 하기 위해)
model.add(Bidirectional(LSTM(units=16,
               activation='tanh')))
#hidden layer
model.add(Dense(units=32,
                activation='relu'))
#output layer
model.add(Dense(units=1,
                activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss = 'binary_crossentropy',
              metrics=['acc'])

#Call back
cp_cb = ModelCheckpoint(filepath='best_model.ckpt',
                        save_weights_only=True,
                        save_best_only=True,
                        monitor = 'val_loss',
                        verbose=1)
es_cb = EarlyStopping(monitor = 'val_loss',
                      patience=3,
                      verbose=1)
history = model.fit(x_data_train,
                    t_data_train,
                    epochs=5,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=[cp_cb,es_cb],
                    verbose=1)

Epoch 1/5
1875/1875 [==============================] - 52s 25ms/step - loss: 0.5102 - acc: 0.7594 - val_loss: 0.3836 - val_acc: 0.8315

Epoch 00001: val_loss improved from inf to 0.38364, saving model to best_model.ckpt
Epoch 2/5
1875/1875 [==============================] - 46s 25ms/step - loss: 0.3488 - acc: 0.8483 - val_loss: 0.3536 - val_acc: 0.8448

Epoch 00002: val_loss improved from 0.38364 to 0.35356, saving model to best_model.ckpt
Epoch 3/5
1875/1875 [==============================] - 47s 25ms/step - loss: 0.3164 - acc: 0.8641 - val_loss: 0.3578 - val_acc: 0.8400

Epoch 00003: val_loss did not improve from 0.35356
Epoch 4/5
1875/1875 [==============================] - 48s 26ms/step - loss: 0.2999 - acc: 0.8712 - val_loss: 0.3491 - val_acc: 0.8471

Epoch 00004: val_loss improved from 0.35356 to 0.34913, saving model to best_model.ckpt
Epoch 5/5
1875/1875 [==============================] - 46s 24ms/step - loss: 0.2894 - acc: 0.8765 - val_loss: 0.3502 - val_acc: 0.8480

Epoch 000

In [61]:
# 학습이 끝난다음..prediction을 할 수 있어요!
# 예측 데이터는 다음과 같은 데이터를 사용해 보세요!

review_sentences = ['내가 만들어도 이것보단 잘 만들겠다.',
                    '너무너무 재미있었습니다. 감사합니다.',
                    '아..내 돈... 돈 아까워 죽을거 같아요',
                    '아나..이것도 영화라고 만들었냐. 무슨 스토리가 산으로 가냐',
                    '감동과 재미가 같이 있는 영화입니다. 훌륭합니다.',
                    '너무너무 재미없다.. 잠와 죽는줄.',
                    '너무너무 재미있다.. 잠이 확깨네.']

df = pd.DataFrame({'document': review_sentences})


# 정규식을 활용해 영어, 한글, 띄어쓰기만 남기고 나머지 특수문자를 제거
df['document'] = df['document'].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]', '')

# 불용어(stopword)제거

okt = Okt()

def word_tokenization(text):
    stop_words = ['는', '을', '를', '이', '가', '의', '던', '고', '하', '다',
                  '은', '에', '들', '지', '게', '도']
    return [word for word in okt.morphs(text) if word not in stop_words]

data_predict = df['document'].apply((lambda x:word_tokenization(x)))

# 4. 각 문장을 숫자 벡터로 변환하여 encoding
data_predict_seq = tokenizer.texts_to_sequences(data_predict)

# 5. 동일한 문장 길이로 정리해야 합니다.
x_data_predict = pad_sequences(data_predict_seq,
                               truncating='post',
                               padding='pre',
                               maxlen=max_length)

# predict
print(model.predict(x_data_predict))

[[0.32474807]
 [0.3263582 ]
 [0.16081178]
 [0.24751401]
 [0.62622243]
 [0.11708194]
 [0.21614842]]


c:\Users\user\anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  
